In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class CustomDense(tf.keras.layers.Layer):
    def __init__(self, units=32, activation=None):
        super(CustomDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
    
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            name='kernel', 
            initial_value=w_init(
                shape=(input_shape[-1], self.units), dtype=tf.float32
            ),
            trainable=True
        )
        
        # Initialoze the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            name='bias',
            initial_value=b_init(
                shape=(self.units,), dtype='float32'
            ),
            trainable=True
        )
        
        super().build(input_shape)
    
    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.w) + self.b)

In [3]:
# declare an instance of the class
my_dense = CustomDense(units=1)

# define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense(x)

# Parameters of the base layer class like `variables` can be used
print(my_dense.variables)

[<tf.Variable 'custom_dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.0190789]], dtype=float32)>, <tf.Variable 'custom_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [4]:
x_train = np.array([1., 2., 3., 4., 5.])
y_train = np.array([1., 4., 9., 16., 25.])

inp = tf.keras.Input((1,))
x = CustomDense(128, activation='relu')(inp)
x = CustomDense(128, activation='relu')(x)
x = CustomDense(128, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)



In [5]:
model = tf.keras.Model(inputs=inp, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
custom_dense_1 (CustomDense) (None, 128)               256       
_________________________________________________________________
custom_dense_2 (CustomDense) (None, 128)               16512     
_________________________________________________________________
custom_dense_3 (CustomDense) (None, 128)               16512     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 33,409
Trainable params: 33,409
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='adam', loss='mae', metrics=['accuracy'])
hisory = model.fit(x_train, y_train, epochs=500, verbose=0)

In [7]:
model.predict([6.])

array([[33.439816]], dtype=float32)

In [8]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [15]:
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_custom_model():
    inp = tf.keras.Input((28, 28))
    x = tf.keras.layers.Flatten()(inp)
    x = CustomDense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = CustomDense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = CustomDense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    output = CustomDense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs=inp, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

def create_model():
    inp = tf.keras.Input((28, 28))
    x = tf.keras.layers.Flatten(input_shape=(28, 28))(inp)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    output = tf.keras.layers.Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs=inp, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

def create_sequential_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        CustomDense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [10]:
model = create_custom_model()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
custom_dense_4 (CustomDense) (None, 256)               200960    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
custom_dense_5 (CustomDense) (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
custom_dense_6 (CustomDense) (None, 64)                8256

In [11]:
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3251 - accuracy: 0.9021
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1426 - accuracy: 0.9588
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1089 - accuracy: 0.9683
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0913 - accuracy: 0.9733
Epoch 5/5
313/313 [==============================] - 0s 938us/step - loss: 0.0771 - accuracy: 0.9785


[0.07714945077896118, 0.9785000085830688]

In [12]:
model = create_model()
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               200960    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256

In [13]:
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3096 - accuracy: 0.9066
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1431 - accuracy: 0.9582
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1121 - accuracy: 0.9676
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0928 - accuracy: 0.9723
Epoch 5/5
313/313 [==============================] - 0s 907us/step - loss: 0.0752 - accuracy: 0.9781


[0.07521041482686996, 0.9781000018119812]

In [16]:
model = create_sequential_model()
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 1.1996 - accuracy: 0.7009
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5323 - accuracy: 0.8536
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4201 - accuracy: 0.8813
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3731 - accuracy: 0.8932
Epoch 5/5
313/313 [==============================] - 0s 902us/step - loss: 0.2928 - accuracy: 0.9156


[0.2927951216697693, 0.9156000018119812]